In [1]:
import pandas as pd
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
from ipywidgets import IntProgress

/home/neerajku/anaconda3/envs/nemo1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/neerajku/anaconda3/envs/nemo1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/neerajku/anaconda3/envs/nemo1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/neerajku/anaconda3/envs/nemo1/lib/pyt

In [58]:
train=pd.read_csv("traindata.csv")

In [74]:
possible_labels = train.Rating.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index



In [75]:
train['label'] = train.Rating.replace(label_dict)

In [76]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train.index.values, 
                                                  train.label.values, 
                                                  test_size=0.3, 
                                                  random_state=42, 
                                                  stratify=train.label.values)




In [77]:
train['data_type'] = ['not_set']*train.shape[0]

train.loc[X_train, 'data_type'] = 'train'
train.loc[X_val, 'data_type'] = 'val'

train.groupby(['Rating', 'label', 'data_type']).count()

ReviewTitle  Review
Rating label data_type                     
1      0     train             1400    1400
             val                600     600
2      1     train             1400    1400
             val                600     600
3      2     train             1400    1400
             val                600     600
4      3     train             1400    1400
             val                600     600
5      4     train             1400    1400
             val                600     600

In [78]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          



In [79]:
df=train

In [80]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Review.values, 
    add_special_tokens=True,truncation=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Review.values, 
    add_special_tokens=True,truncation=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

In [81]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [82]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [83]:
model = model.cuda()

In [84]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 4

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [85]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [86]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    
    
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    
    
    
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]

       
        acc = len(y_preds[y_preds==label])/len(y_true)
        

        return acc



In [87]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import random
import numpy as np


seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [88]:

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), 'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    val_acc = accuracy_per_class(predictions, true_vals)
    
    print(val_loss,val_f1,val_acc)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    tqdm.write(f'Accuracy: {val_acc}')
    



  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/1750 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.1926133090257645
0
1.080801718155543 0.5357329317077988 0.5316666666666666
Validation loss: 1.080801718155543
F1 Score (Weighted): 0.5357329317077988
Accuracy: 0.5316666666666666


Epoch 2:   0%|          | 0/1750 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.9020630540251732
0
1.123792999337117 0.5507810673282568 0.65
Validation loss: 1.123792999337117
F1 Score (Weighted): 0.5507810673282568
Accuracy: 0.65


...


Epoch 3
Training loss: 0.6918127651427474
0
1.3394774663299323 0.5465856528949405 0.5883333333333334
Validation loss: 1.3394774663299323
F1 Score (Weighted): 0.5465856528949405
Accuracy: 0.5883333333333334


Epoch 4:   0%|          | 0/1750 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.5199444559056844
0
1.6497163189140458 0.5315594548438232 0.6483333333333333
Validation loss: 1.6497163189140458
F1 Score (Weighted): 0.5315594548438232
Accuracy: 0.6483333333333333


Epoch 5:   0%|          | 0/1750 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.3996953455039433
0
1.8668909998827925 0.5378003573896101 0.5916666666666667
Validation loss: 1.8668909998827925
F1 Score (Weighted): 0.5378003573896101
Accuracy: 0.5916666666666667
